Indicators of interest:
-	Percentage of health facility having direct access to an all season road.
-	Percentage of health facility within 2km of an all season road.
-	Percentage of population within 2h of driving to the nearest primary care facility (population level, and by SES quintile).
-	Percentage of population within 2h of driving to the nearest district hospital (population, and by SES quintile).


In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
import rasterio as rio
import numpy as np
from shapely.geometry import Point
import skimage.graph as graph

In [2]:
sys.path.append('/home/wb514197/Repos/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
sys.path.append('/home/wb514197/Repos/GOSTnets') # it also depends on gostnets for some reason
sys.path.append('/home/wb514197/Repos/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [3]:
import GOSTRocks.rasterMisc as rMisc
import GOSTNetsRaster.market_access as ma
from infrasap import aggregator
from rasterstats import zonal_stats

no xarray


In [4]:
iso3 = 'LBR'

In [5]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [6]:
global_admin2 = '/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp'
adm2 = gpd.read_file(global_admin2)
adm2 = adm2.loc[adm2.ISO3==iso3].copy()
adm2 = adm2.to_crs("EPSG:4326")

In [7]:
adm2.reset_index(inplace=True)

In [8]:
tt_health_rio = rio.open(os.path.join(out_folder, "tt_health.tif"))
tt_health = tt_health_rio.read(1, masked=True)

In [9]:
tt_hospital_rio = rio.open(os.path.join(out_folder, "tt_hospital.tif"))
tt_hospital = tt_hospital_rio.read(1, masked=True)

In [10]:
out_pop_surface_std = os.path.join(out_folder, "WP_2020_1km_STD.tif")
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)

In [11]:
pop_120_heatlh = pop*(tt_health<=120)
pop_120_hospital = pop*(tt_hospital<=120)

In [12]:
zs_pop = pd.DataFrame(zonal_stats(adm2, pop.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop'})
zs_lt_120_health = pd.DataFrame(zonal_stats(adm2, pop_120_heatlh.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop_120_health'})
zs_lt_120_hospital = pd.DataFrame(zonal_stats(adm2, pop_120_hospital.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop_120_hospital'})

In [13]:
zs = zs_pop.join(zs_lt_120_health).join(zs_lt_120_hospital)

In [14]:
zs.loc[:, "health_pct"] = zs.loc[:, "pop_120_health"]/zs.loc[:, "pop"]
zs.loc[:, "hospital_pct"] = zs.loc[:, "pop_120_hospital"]/zs.loc[:, "pop"]

In [15]:
res = adm2.join(zs)

Roads

In [17]:
from infrasap import osm_extractor as osm
import json
from utm_zone import epsg as epsg_get

In [298]:
roads = gpd.read_file(os.path.join(input_dir, 'osm', 'highways.shp'))

In [299]:
adm2_json = json.loads(adm2.to_json())
epsg = epsg_get(adm2_json)

In [300]:
roads = roads.to_crs(epsg)

In [301]:
roads['geometry'] = roads['geometry'].apply(lambda x: x.buffer(100)) # 2000

In [302]:
osm.OSMLR_Classes

{'motorway': 'OSMLR level 1',
 'motorway_link': 'OSMLR level 1',
 'trunk': 'OSMLR level 1',
 'trunk_link': 'OSMLR level 1',
 'primary': 'OSMLR level 1',
 'primary_link': 'OSMLR level 1',
 'secondary': 'OSMLR level 2',
 'secondary_link': 'OSMLR level 2',
 'tertiary': 'OSMLR level 2',
 'tertiary_link': 'OSMLR level 2',
 'unclassified': 'OSMLR level 3',
 'unclassified_link': 'OSMLR level 3',
 'residential': 'OSMLR level 3',
 'residential_link': 'OSMLR level 3',
 'track': 'OSMLR level 4',
 'service': 'OSMLR level 4'}

In [303]:
roads['OSMLR'] = roads['type'].map(osm.OSMLR_Classes)

In [304]:
def get_num(x):
    try:
        return(int(x))
    except:
        return(5)
roads['OSMLR_num'] = roads['OSMLR'].apply(lambda x: get_num(str(x)[-1]))

In [305]:
lbr_master = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))
lbr_master['status'] = lbr_master['Status'].apply(lambda x: x.lower())
lbr_master['hf_type'] = lbr_master['HF Type'].apply(lambda x: x.lower())
lbr = lbr_master.loc[lbr_master.Status!="non-functional"].copy()
lbr = lbr.loc[~lbr.Lat.isna()].copy()
geoms = [Point(xy) for xy in zip(lbr.Long, lbr.Lat)]
lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)

In [306]:
global_admin = '/home/public/Data/GLOBAL/ADMIN/g2015_0_simplified.shp'
adm0 = gpd.read_file(global_admin)
aoi = adm0.loc[adm0.ISO3166_1_==iso3]

In [307]:
lbr_geo_filt = lbr_geo.loc[lbr_geo.intersects(aoi.unary_union)]

In [308]:
lbr_geo_filt.reset_index(inplace=True, drop=True)

In [309]:
hospitals = lbr_geo_filt.loc[lbr_geo_filt.hf_type=="hospital"].copy()

In [310]:
hospitals.reset_index(inplace=True, drop=True)

In [330]:
lbr_geo_filt[['hf_type']].value_counts()

hf_type      
clinic           527
health center     53
hospital          35
dtype: int64

In [311]:
len(lbr_geo_filt), len(hospitals)

(615, 35)

In [329]:
lbr_geo_filt.to_file(os.path.join(out_folder, "LBR_health.geojson"), driver='GeoJSON')

In [312]:
lbr_geo_filt = lbr_geo_filt.to_crs(epsg)
hospitals = hospitals.to_crs(epsg)

In [313]:
roads_1 = roads.loc[roads.OSMLR_num<=1].unary_union
roads_2 = roads.loc[roads.OSMLR_num<=2].unary_union
roads_3 = roads.loc[roads.OSMLR_num<=3].unary_union
roads_4 = roads.loc[roads.OSMLR_num<=4].unary_union

In [314]:
lbr_geo_filt.loc[:, "bool_1"] = lbr_geo_filt.intersects(roads_1)
lbr_geo_filt.loc[:, "bool_2"] = lbr_geo_filt.intersects(roads_2)
lbr_geo_filt.loc[:, "bool_3"] = lbr_geo_filt.intersects(roads_3)
lbr_geo_filt.loc[:, "bool_4"] = lbr_geo_filt.intersects(roads_4)

In [315]:
lbr_geo_filt = lbr_geo_filt.to_crs(adm2.crs)

In [316]:
facilities = gpd.sjoin(lbr_geo_filt, adm2[['OBJECTID', 'WB_ADM2_CO', 'geometry']], how='left')

In [317]:
res_osmlr = facilities[['bool_1','bool_2','bool_3','bool_4','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()

In [318]:
res_count = facilities[['WB_ADM2_CO','bool_1']].groupby('WB_ADM2_CO').count().rename(columns={'bool_1':'count'})

In [319]:
res_osmlr_pct = res_osmlr.apply(lambda x: x/res_count['count'])

In [320]:
res_osmlr = res_osmlr.join(res_count)

In [321]:
res_osmlr = res_osmlr.join(res_osmlr_pct, rsuffix="_pct")

In [322]:
# res_osmlr.loc[:, 'osmlr1_pct'] = res_osmlr.loc[:, 'bool_1']/res_osmlr.loc[:, 'count']
# res_osmlr.loc[:, 'osmlr2_pct'] = res_osmlr.loc[:, 'bool_2']/res_osmlr.loc[:, 'count']
# res_osmlr.loc[:, 'osmlr3_pct'] = res_osmlr.loc[:, 'bool_3']/res_osmlr.loc[:, 'count']
# res_osmlr.loc[:, 'osmlr4_pct'] = res_osmlr.loc[:, 'bool_4']/res_osmlr.loc[:, 'count']

In [323]:
res_merge = res.merge(res_osmlr, left_on="WB_ADM2_CO", right_index=True)

In [324]:
res_merge.head()

,index,OBJECTID,ISO_A2,WB_ADM1_CO,WB_ADM0_CO,WB_ADM0_NA,WB_ADM1_NA,WB_ADM2_CO,WB_ADM2_NA,Shape_Leng,...,hospital_pct,bool_1,bool_2,bool_3,bool_4,count,bool_1_pct,bool_2_pct,bool_3_pct,bool_4_pct
0,22303,22304,LR,1814,144,Liberia,Bomi,39172,Klay,230304.410707,...,0.999539,2,10,28,31,32,0.062500,0.3125,0.875000,0.968750
1,22304,22305,LR,1814,144,Liberia,Bomi,39173,Mecca,88135.540238,...,1.000000,0,0,1,1,1,0.000000,0.0000,1.000000,1.000000
2,22305,22306,LR,1815,144,Liberia,Bong,39174,Fuamah,140545.898902,...,0.855267,0,0,2,2,3,0.000000,0.0000,0.666667,0.666667
3,22306,22307,LR,1815,144,Liberia,Bong,39175,Jorquelleh,186576.711754,...,0.991790,2,2,5,5,5,0.400000,0.4000,1.000000,1.000000
4,22307,22308,LR,1815,144,Liberia,Bong,39176,Kokoyah,178374.092081,...,0.928034,5,6,6,6,6,0.833333,1.0000,1.000000,1.000000


In [325]:
res_merge.to_file(os.path.join(out_folder, "health_roads_100m.shp"), driver="ESRI Shapefile")

/home/wb514197/.conda/envs/graph/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [297]:
res_merge.to_file(os.path.join(out_folder, "health_roads_2km.shp"), driver="ESRI Shapefile")

/home/wb514197/.conda/envs/graph/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### FB Analysis

In [18]:
input_dir

'/home/public/Data/PROJECTS/Health'

In [19]:
fb = pd.read_csv(os.path.join(input_dir, 'facebook', 'lbr_relative_wealth_index.csv'))

In [20]:
fb_geoms = [Point(xy) for xy in zip(fb.longitude, fb.latitude)]
fb_geo = gpd.GeoDataFrame(fb, crs='EPSG:4326', geometry=fb_geoms)

In [174]:
fb_zs = pd.DataFrame(zonal_stats(fb_geo, tt_health.filled(), affine=tt_health_rio.transform, stats='mean', nodata=tt_health_rio.nodata)).rename(columns={'mean':'tt_health'})
fb_zs_hosp = pd.DataFrame(zonal_stats(fb_geo, tt_hospital.filled(), affine=tt_hospital_rio.transform, stats='mean', nodata=tt_hospital_rio.nodata)).rename(columns={'mean':'tt_hospital'})

In [176]:
fb_geo = fb_geo.join(fb_zs).join(fb_zs_hosp)

In [177]:
fb_geo

,latitude,longitude,rwi,error,geometry,tt_health,tt_hospital
0,4.576425,-7.987061,-0.902,0.295,POINT (-7.98706 4.57642),131.144119,185.625931
1,6.217012,-10.228271,-0.082,0.289,POINT (-10.22827 6.21701),7.249699,39.488678
2,5.626919,-8.316650,-0.517,0.286,POINT (-8.31665 5.62692),33.827759,146.152924
3,6.544560,-9.656982,-0.489,0.267,POINT (-9.65698 6.54456),38.609650,103.398010
4,7.852499,-9.656982,-0.761,0.260,POINT (-9.65698 7.85250),46.817535,74.537605
...,...,...,...,...,...,...,...
6184,7.111795,-8.822021,-0.277,0.271,POINT (-8.82202 7.11179),5.380243,35.791603
6185,6.238855,-9.547119,-0.942,0.285,POINT (-9.54712 6.23886),161.047684,180.021103
6186,6.369894,-9.964600,-0.279,0.269,POINT (-9.96460 6.36989),40.256237,92.006248
6187,6.806444,-9.613037,-0.457,0.268,POINT (-9.61304 6.80644),21.252628,52.030640


In [185]:
fb_geo.loc[:, "rwi_cut"] = pd.qcut(fb_geo['rwi'], [0, .2, .4, .6, .8, 1.], labels=['lowest', 'second-lowest', 'middle', 'second-highest', 'highest'])

In [200]:
# fb_geo.groupby()

In [188]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from pyquadkey2 import quadkey

In [218]:
fb_geo = gpd.sjoin(fb_geo, adm2[['WB_ADM2_CO', 'WB_ADM2_NA', 'geometry']])

In [242]:
# not sure why this doesn't work aint nobody got time for that
def get_point_in_polygon(lat, lon, polygons):
    """ 
    @param lat: double
    @param lon: double
    @param polygons: dict
    @return geo_id: str
    """
    point = Point(lon, lat)
    for geo_id in polygons:
        polygon = polygons[geo_id]
        if polygon.contains(point):
            return geo_id
        else:
            return 'null'

shapefile = adm2.copy()
polygons = dict(zip(shapefile['WB_ADM2_CO'], shapefile['geometry']))
print(shapefile.shape)
shapefile.head()

fb_geo['geo_id'] = fb_geo.apply(lambda x: get_point_in_polygon(x['latitude'], x['longitude'], polygons), axis=1)

(64, 13)


In [227]:
population = pd.read_csv(os.path.join(input_dir, 'facebook', 'population_lbr_2019-07-01.csv'))

In [228]:
population = population.rename(columns={'Population': 'pop_2020'})

In [231]:
population['quadkey'] = population.apply(lambda x: str(quadkey.from_geo((x['Lat'], x['Lon']), 14)), axis=1)

In [233]:
bing_tile_z14_pop = population.groupby('quadkey', as_index=False)['pop_2020'].sum()

In [238]:
fb_geo['quadkey'] = fb_geo.apply(lambda x: str(quadkey.from_geo((x['latitude'], x['longitude']), 14)), axis=1)

In [244]:
rwi = fb_geo.merge(bing_tile_z14_pop[['quadkey', 'pop_2020']], on='quadkey', how='inner')

In [247]:
rwi.loc[:, "tt_health_bool"] = rwi.tt_health<=120
rwi.loc[:, "tt_hospital_bool"] = rwi.tt_hospital<=120

In [252]:
rwi.columns

Index(['latitude', 'longitude', 'rwi', 'error', 'geometry', 'tt_health',
       'tt_hospital', 'rwi_cut', 'geo_id', 'index_right', 'WB_ADM2_CO',
       'WB_ADM2_NA', 'quadkey', 'pop_2020', 'tt_health_bool',
       'tt_hospital_bool'],
      dtype='object')

In [265]:
# rwi[['WB_ADM2_NA', 'tt_health_bool', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'tt_health_bool', 'rwi_cut']).sum()
# res_rwi = rwi[['WB_ADM2_NA', 'tt_health_bool', 'tt_hospital_bool', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'tt_health_bool', 'tt_hospital_bool', 'rwi_cut']).sum()

In [273]:
rwi_pop = rwi[['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum()

In [274]:
rwi_health = rwi.loc[rwi.tt_health_bool==True, ['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_health'})

In [275]:
rwi_hospital = rwi.loc[rwi.tt_hospital_bool==True, ['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_hospital'})

In [276]:
res_rwi = rwi_pop.join(rwi_health).join(rwi_hospital)

In [281]:
res_rwi.loc[:, "health_pct"] = res_rwi['pop_120_health']/res_rwi['pop_2020']
res_rwi.loc[:, "hospital_pct"] = res_rwi['pop_120_hospital']/res_rwi['pop_2020']

In [284]:
res_rwi.head()

pop_2020  pop_120_health  pop_120_hospital  \
WB_ADM2_NA rwi_cut                                                          
Barrobo    lowest           8450.701912     8450.701912       8174.211583   
           second-lowest    7407.226104     6670.420840       6509.673563   
           middle          10500.503335    10500.503335      10500.503335   
           second-highest   5003.333989     3959.763720       3569.685334   
           highest         13804.978271    12536.130601      12536.130601   

                           health_pct  hospital_pct  
WB_ADM2_NA rwi_cut                                   
Barrobo    lowest            1.000000      0.967282  
           second-lowest     0.900529      0.878827  
           middle            1.000000      1.000000  
           second-highest    0.791425      0.713461  
           highest           0.908088      0.908088

In [285]:
res_rwi.to_csv(os.path.join(out_folder, 'test_rwi.csv'))

In [286]:
rwi_pop_adm0 = rwi[['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum()

In [288]:
rwi_health_adm0 = rwi.loc[rwi.tt_health_bool==True, ['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_health'})

In [289]:
rwi_hospital_adm0 = rwi.loc[rwi.tt_hospital_bool==True, ['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_hospital'})

In [291]:
res_rwi_adm0 = rwi_pop_adm0.join(rwi_health_adm0).join(rwi_hospital_adm0)

In [292]:
res_rwi_adm0.loc[:, "health_pct"] = res_rwi_adm0['pop_120_health']/res_rwi_adm0['pop_2020']
res_rwi_adm0.loc[:, "hospital_pct"] = res_rwi_adm0['pop_120_hospital']/res_rwi_adm0['pop_2020']

In [294]:
res_rwi_adm0.to_csv(os.path.join(out_folder, 'rwi_adm0.csv'))